# Analytical solution Prandle and Heaps: Vertical velocity structure 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cmath
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, fixed

### Prandle: time-dependent analytical solution for tidal-induced currents

In [2]:
def Prandle(t, z, N):
    
        #Coast Characteristics
      
    zeta = 0.2    
    h = 25
    k = 0.0025
    f = 1.09083078 * 10**-4
    w = 1.4 * 10**-4
    g = 9.81
    drho_dx = - 2 * 10**-4
    rho = 1000
    w = 1.4 * 10**-4
    U = 0.43
    s = (8 * k * U) / (3 * np.pi * N)
    
        #Prandle
    
    a_plus = (1 + complex(0, 1)) * cmath.sqrt((f + w) / (2 * N))
    R_plus = np.cosh(a_plus * (z - h)) - np.cosh(a_plus * h) - (a_plus / s) * np.sinh(a_plus * h) 
    R_plus_avg =  - np.cosh(a_plus * h) + (1 / (a_plus * h) - (a_plus / s)) * np.sinh(a_plus * h)

    a_min = (1 + complex(0, 1)) * cmath.sqrt((f - w) / (2 * N))
    R_min = np.cosh(a_min * (z - h)) - np.cosh(a_min * h) - (a_min / s) * np.sinh(a_min * h) 
    R_min_avg =  - np.cosh(a_min * h) + (1 / (a_min * h) - (a_min / s)) * np.sinh(a_min * h)

    R_plus_plot = R_plus / R_plus_avg
    R_min_plot = R_min / R_min_avg 
    
    R_plus_amp = np.sqrt(R_plus_plot.real**2 + R_plus_plot.imag**2) * U
    R_min_amp = np.sqrt(R_min_plot.real**2 + R_min_plot.imag**2) * U

    R_plus_time = R_plus_amp * np.exp(-complex(0, 1) * w * t) 
    R_min_time = R_min_amp * np.exp(complex(0, 1) * w * t)
    R = R_plus_time + R_min_time
    
    u_p = R.imag
    v_p = R.real
    
        #Plot
    
    x = np.zeros(len(z))
    y = np.zeros(len(z))
    
    fig = plt.figure(figsize=(16,10))
    ax = fig.add_subplot(projection='3d')
    
    ax.quiver(x, y, z, u_p, v_p, 0, label='Prandle')
    ax.axis(xmin=-.3, xmax=.3)
    ax.axis(ymin=-1, ymax=1)
    ax.title(f'Vertical eddy viscosity = {N} [m/, total depth = {h})
    ax.set_xlabel('Cross-shore velocity [m/s]')
    ax.set_ylabel('Alongshore velocity [m/s]')
    ax.legend(loc='upper left')

In [3]:
z = np.linspace(0, 25, 100)
N = 0.0028
t = np.linspace(0, 89400, 100)
interact(Prandle, t=(0, 89400, 89400/100), z=fixed(z), N=fixed(N))

interactive(children=(FloatSlider(value=44700.0, description='t', max=89400.0, step=894.0), Output()), _dom_cl…

<function __main__.Prandle(t, z, N)>

### Heaps: time-independent analytical solution for residual density-driven currents

In [4]:
def Heaps(z, N):
    
        #Coast Characteristics
      
    zeta = 0.2    
    h = 25
    k = 0.0025
    f = 1.09083078 * 10**-4
    w = 1.4 * 10**-4
    g = 9.81
    drho_dx = - 2 * 10**-4
    rho = 1000
    w = 1.4 * 10**-4
    U = 0.43
    s = (8 * k * U) / (3 * np.pi * N)
    
        #Heaps
        
    D = np.pi * ((2 * N)/ f)**0.5
    Z = z + zeta
    H = h + zeta
    eta = Z / H
    a = np.pi * H / D
    a1 = a * (1 - eta)
    a2 = a * eta
    b = k * (H / N)
    C = a * (np.sinh(a) * np.cos(a) - np.cosh(a) * np.sin(a)) + b * np.cosh(a) * np.cos(a)
    E = a * (np.sinh(a) * np.cos(a) + np.cosh(a) * np.sin(a)) + b * np.sinh(a) * np.sin(a)
    L = b * np.cosh(a2) * np.cos(a2)
    M = b * np.sinh(a2) * np.sin(a2)
    P = C / (C**2 + E**2)
    Q = E / (C**2 + E**2)
    R = P * np.cosh(a) * np.cos(a) + Q * np.sinh(a) * np.sin(a)
    S = 1 - (R * b)
    Delta = (R - P - S) / S
    Lambda = 1 + b + b * Delta
    X = np.cosh(a1) * np.cos(a1) + (b / (2 * a)) * (np.sinh(a1) * np.cos(a1) + np.cosh(a1) * np.sin(a1)) - Lambda * np.cosh(a2) * np.cos(a2)
    Y = np.sinh(a1) * np.sin(a1) + (b / (2 * a)) * (np.cosh(a1) * np.sin(a1) - np.sinh(a1) * np.cos(a1)) - Lambda * np.sinh(a2) * np.sin(a2)
    
    u_h = ((g * H) / f) * (X * Q - Y * P) * (drho_dx) / rho
    v_h = ((g * H) / f) * (X * P + Y * Q + Delta + eta) * (drho_dx) / rho  
    
    u_h = np.flip(u_h)
    v_h = np.flip(v_h)
    
        #Plot
    
    x = np.zeros(len(z))
    y = np.zeros(len(z))
    
    fig = plt.figure(figsize=(16, 10))  
    ax = fig.add_subplot(projection='3d')

    ax.quiver(x, y, z, u_h, v_h, 0, label='Heaps')    
    ax.axis(xmin=-.3, xmax=.3)
    ax.axis(ymin=-1, ymax=1)
    ax.set_xlabel('Cross-shore velocity [m/s]')
    ax.set_ylabel('Alongshore velocity [m/s]')
    ax.legend(loc='upper left')

In [5]:
z = np.linspace(0, 25, 100)
N = 0.0028
interact(Heaps, N=(0.00001, 0.1, 0.001), z=fixed(z))

interactive(children=(FloatSlider(value=0.049010000000000005, description='N', max=0.1, min=1e-05, step=0.001)…

<function __main__.Heaps(z, N)>

### Prandle and Heaps combined

In [6]:
def Prandle_Heaps(t, z, N):
    
        #Coast Characteristics
      
    zeta = 0.2    
    h = 25
    k = 0.0025
    f = 1.09083078 * 10**-4
    w = 1.4 * 10**-4
    g = 9.81
    drho_dx = - 2 * 10**-4
    rho = 1000
    w = 1.4 * 10**-4
    U = 0.43
    s = (8 * k * U) / (3 * np.pi * N)
    
        #Heaps
        
    D = np.pi * ((2 * N)/ f)**0.5
    Z = z + zeta
    H = h + zeta
    eta = Z / H
    a = np.pi * H / D
    a1 = a * (1 - eta)
    a2 = a * eta
    b = k * (H / N)
    C = a * (np.sinh(a) * np.cos(a) - np.cosh(a) * np.sin(a)) + b * np.cosh(a) * np.cos(a)
    E = a * (np.sinh(a) * np.cos(a) + np.cosh(a) * np.sin(a)) + b * np.sinh(a) * np.sin(a)
    L = b * np.cosh(a2) * np.cos(a2)
    M = b * np.sinh(a2) * np.sin(a2)
    P = C / (C**2 + E**2)
    Q = E / (C**2 + E**2)
    R = P * np.cosh(a) * np.cos(a) + Q * np.sinh(a) * np.sin(a)
    S = 1 - (R * b)
    Delta = (R - P - S) / S
    Lambda = 1 + b + b * Delta
    X = np.cosh(a1) * np.cos(a1) + (b / (2 * a)) * (np.sinh(a1) * np.cos(a1) + np.cosh(a1) * np.sin(a1)) - Lambda * np.cosh(a2) * np.cos(a2)
    Y = np.sinh(a1) * np.sin(a1) + (b / (2 * a)) * (np.cosh(a1) * np.sin(a1) - np.sinh(a1) * np.cos(a1)) - Lambda * np.sinh(a2) * np.sin(a2)
    
    u_h = ((g * H) / f) * (X * Q - Y * P) * (drho_dx) / rho
    v_h = ((g * H) / f) * (X * P + Y * Q + Delta + eta) * (drho_dx) / rho  
    
    u_h = np.flip(u_h)
    v_h = np.flip(v_h)
    
        #Prandle
    
    a_plus = (1 + complex(0, 1)) * cmath.sqrt((f + w) / (2 * N))
    R_plus = np.cosh(a_plus * (z - h)) - np.cosh(a_plus * h) - (a_plus / s) * np.sinh(a_plus * h) 
    R_plus_avg =  - np.cosh(a_plus * h) + (1 / (a_plus * h) - (a_plus / s)) * np.sinh(a_plus * h)

    a_min = (1 + complex(0, 1)) * cmath.sqrt((f - w) / (2 * N))
    R_min = np.cosh(a_min * (z - h)) - np.cosh(a_min * h) - (a_min / s) * np.sinh(a_min * h) 
    R_min_avg =  - np.cosh(a_min * h) + (1 / (a_min * h) - (a_min / s)) * np.sinh(a_min * h)

    R_plus_plot = R_plus / R_plus_avg
    R_min_plot = R_min / R_min_avg 
    
    R_plus_amp = np.sqrt(R_plus_plot.real**2 + R_plus_plot.imag**2) * U
    R_min_amp = np.sqrt(R_min_plot.real**2 + R_min_plot.imag**2) * U

    R_plus_time = R_plus_amp * np.exp(-complex(0, 1) * w * t) 
    R_min_time = R_min_amp * np.exp(complex(0, 1) * w * t)
    R = R_plus_time + R_min_time
    
    u_p = R.imag
    v_p = R.real
    
        #Prandle + Heaps
    
    u = u_h + u_p
    v = v_h + v_p
    
        #Plot
    
    x = np.zeros(len(z))
    y = np.zeros(len(z))
    
    fig = plt.figure(figsize=(16, 10))  
    ax = fig.add_subplot(projection='3d')
    
    ax.quiver(x, y, z, u, v, 0, label='Prandle + Heaps') 
    ax.axis(xmin=-.3, xmax=.3)
    ax.axis(ymin=-1, ymax=1)
    ax.set_xlabel('Cross-shore velocity [m/s]')
    ax.set_ylabel('Alongshore velocity [m/s]')
    ax.legend(loc='upper left')

In [7]:
z = np.linspace(0, 25, 100)
N = 0.0028
t = np.linspace(0, 89400, 100)
interact(Prandle_Heaps, t=(0, 89400, 89400/100), z=fixed(z), N=fixed(N))

interactive(children=(FloatSlider(value=44700.0, description='t', max=89400.0, step=894.0), Output()), _dom_cl…

<function __main__.Prandle_Heaps(t, z, N)>